# Using SPARQL Endpoints with PyShEx

The '-ss' option tells `shex_evaluator` to reference SPARQL endpoint instead of a local RDF graph.  As ab example:

```bash
shexeval -ss  https://query.wikidata.org/sparql test_shape.shex
```

Would treat the *entire* contents of the Wikidata sparql endpoint as one huge graph.

This is probably not what you want to do.  There are several options that can be used in its place:
1.  Evaluate every non-bnode subject:

    `shexeval -ss -A http://wifo5-04.informatik.uni-mannheim.de/drugbank/sparql test.shex`
    
    
2. Use the RDF type ("-ut") to identify the shape expressions(s) to evaluate the nodes against

    `shexeval -ss -ut http://wifo5-04.informatik.uni-mannheim.de/drugbank/vocab/resource/class/Offer http://wifo5-04.informatik.uni-mannheim.de/drugbank/sparql test.shex`
    
    
3. The shape to evaluate is the object of a specific predicate ("-sp")

    `shexeval -ss -sp http://wifo5-04.informatik.uni-mannheim.de/drugbank/resource/drugbank/  http://wifo5-04.informatik.uni-mannheim.de/drugbank/sparql test.shex`
    
The most workable approach, however, is to use a SPARQL query to generate the focus nodes.  The query must yield a list of elements named `?item`:

```bash
> cat q1.sparql
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX vocabClass: <http://wifo5-04.informatik.uni-mannheim.de/drugbank/vocab/resource/class/>

SELECT DISTINCT ?item WHERE {
  ?item rdf:type vocabClass:Offer
}
LIMIT 10
>
```

This query can then be used as input to the ShEx evaluator:
    
    `shexeval -se -ss --stopafter -sq q1.sparql http://wifo5-04.informatik.uni-mannheim.de/drugbank/sparql test.shex`
    
    "-se" option says stop on the first error
    
    "--stopafter" option says stop after N (5) nodes are evaluated
    
    

## SPARQL Endpoints in Python

This example assumes that PyShEx has been installed in jupyter environment

In [1]:
from pyshex.shex_evaluator import ShExEvaluator
from pyshex.user_agent import SlurpyGraphWithAgent
from pyshex.utils.sparql_query import SPARQLQuery

# SPARQL Endpoint
endpoint = 'http://wifo5-04.informatik.uni-mannheim.de/drugbank/sparql'

# SPARQL Query
sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX vocabClass: <http://wifo5-04.informatik.uni-mannheim.de/drugbank/vocab/resource/class/>

SELECT DISTINCT ?item WHERE {
  ?item rdf:type vocabClass:Offer
}
LIMIT 20
"""

# ShEx Expression
shex = """
PREFIX drugbank: <http://wifo5-04.informatik.uni-mannheim.de/drugbank/resource/drugbank/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX : <http://example.org/t1/>

START=@:S1

:S1 {foaf:page IRI{3} ;                     # one or more foaf pages
     drugbank:limsDrugId xsd:string       # ane exactly one drug id
}"""


result = ShExEvaluator(SlurpyGraphWithAgent(endpoint),
                       shex,
                       SPARQLQuery(endpoint, sparql).focus_nodes()).evaluate()
for r in result:
    print(f"{r.focus}: ", end="")
    if not r.result:
        print(f"FAIL: {r.reason}")
    else:
        print("PASS")


http://wifo5-04.informatik.uni-mannheim.de/drugbank/resource/drugs/DB00001: PASS
http://wifo5-04.informatik.uni-mannheim.de/drugbank/resource/drugs/DB00002: PASS
http://wifo5-04.informatik.uni-mannheim.de/drugbank/resource/drugs/DB00003: PASS
http://wifo5-04.informatik.uni-mannheim.de/drugbank/resource/drugs/DB00004: PASS
http://wifo5-04.informatik.uni-mannheim.de/drugbank/resource/drugs/DB00005: PASS
http://wifo5-04.informatik.uni-mannheim.de/drugbank/resource/drugs/DB00006: PASS
http://wifo5-04.informatik.uni-mannheim.de/drugbank/resource/drugs/DB00007: PASS
http://wifo5-04.informatik.uni-mannheim.de/drugbank/resource/drugs/DB00008: PASS
http://wifo5-04.informatik.uni-mannheim.de/drugbank/resource/drugs/DB00009: PASS
http://wifo5-04.informatik.uni-mannheim.de/drugbank/resource/drugs/DB00010: PASS
http://wifo5-04.informatik.uni-mannheim.de/drugbank/resource/drugs/DB00011: FAIL:   Testing <http://wifo5-04.informatik.uni-mannheim.de/drugbank/resource/drugs/DB00011> against shape http://